In [ ]:
import sklearn
from typing import List, Any
from openai import AzureOpenAI
import os
os.environ["no_proxy"] = "10.156.254.10"
import openai
openai.api_key = "dtnumds"
openai.api_base = "http://10.156.254.10:8000/v1"
os.environ["OPENAI_API_KEY"] = "dtnumds"
os.environ["OPENAI_API_BASE"] = "http://10.156.254.10:8000/v1"

In [ ]:
client = AzureOpenAI( api_key="dtnumds",
                azure_endpoint="http://10.156.254.10:8000/v1",
                api_version = "2023-07-01-preview" )

In [ ]:
for model in client.models.list().data :
    print(model.id)

In [ ]:
import random
from tqdm import tqdm 

In [ ]:

def compute_embedding(docs: List[str], model: str) -> List[Any]:
    """This function computes embeddings for a given list of texts

    Args:
        docs (List[str]): Input texts to embed
        model (str): Embedding model to used

    Returns:
        _type_: _description_
    """
    embeddings = client.embeddings.create(model=model, input=docs)
    return [e.embedding for e in embeddings.data]

In [ ]:
N_EXAMPLE = 500

In [ ]:
import enchant


In [ ]:
with open("liste.de.mots.francais.frgut.txt", 'r') as f:
    data = f.read()

In [ ]:
fr_dict = data.split('\n')
import re
def generate_random_word():
    word = ""
    while len(word) < 6:
        word = fr_dict[random.randint(0,len(fr_dict))]
    return re.sub('\n', '', word)

In [ ]:
# Automatic labelled generation 
generation_answer_pt = """[INST]
Ta mission est de généré des réponses de chat synthétiques en français. L'objectif est de pouvoir par la suite entrâiner un modèle.  Tu te baseras sur le mot de vocabulaire suivant :
{random_word}
Tu renverras des nouvelles réponses rédigées en français à partir du mots de vocabulaire 
[/INST]
"""
prompts = [generation_answer_pt.format(random_word = generate_random_word()) for _ in range((N_EXAMPLE))]
exemple_reponse = [r.text.split("\n")[1] for r in client.completions.create(prompt=prompts, model="mistral-7b-awq", temperature=0.0, max_tokens=256).choices]

In [ ]:
generate_no_context_prompt = """
Ta mission est de générer de maximum de formulation possible pour exprimer l'absence d'informations dans un context donnée.
Par exemple :
- Je ne dispose pas d'informations dans le contexte fourni pour répondre à la question concernant le diamètre de la Terre.
- Le contexte suivante ne donne aucune information à propos de la taille de la ville de Paris. 
"""

In [ ]:
no_context_example = [c if len(c) > 10 else "vide" for c in client.completions.create(prompt=generate_no_context_prompt,model="mixtral-instruct", temperature=0.0, max_tokens=32000).choices[0].text.split('\n')]

In [ ]:
no_context_example += [c if len(c) > 10 else "vide" for c in client.completions.create(prompt=generate_no_context_prompt,model="mixtral-instruct", temperature=0.1, max_tokens=4096).choices[0].text.split('\n')]

In [ ]:
import pandas as pd 
exemple_reponse_df = pd.DataFrame(exemple_reponse, columns = ['text'])
exemple_reponse_df['label']  = 0
exemple_no_context_df = pd.DataFrame(no_context_example, columns = ['text'])
exemple_no_context_df['label'] = 1
exemple_no_context_df = exemple_no_context_df.loc[exemple_no_context_df['text'] != "vide"]

df = pd.concat((exemple_no_context_df, exemple_reponse_df)).reset_index(drop=True).sample( frac = 1)

In [ ]:
df.to_csv('../../../data/train_data_clf.csv', index=False)

In [ ]:
df.label.value_counts()

In [ ]:
embs  = compute_embedding(df['text'].to_list(), model='dgfip-e5-large')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(embs, df['label'], test_size=0.33, random_state=432)

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier(max_depth=3).fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)

In [ ]:
acc

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
from skops.io import dump, load
serialized = dump(clf, "../../ai_models/clf_pr.skops")
loaded = load("../../ai_models/clf_pr.skops", trusted=True)